# Step 1: Mounting Google Drive and Importing Libraries


In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/grpo-verified-reasoner
!ls

Mounted at /content/drive
/content/drive/MyDrive/grpo-verified-reasoner
data			      LICENSE	 outputs    unsloth_compiled_cache
grpo_trainer_lora_model       models	 README.md  _unsloth_sentencepiece_temp
huggingface_tokenizers_cache  notebooks  src	    wandb


In [2]:
# Install UV (Faster pip)
!pip install --upgrade -qqq uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 55.5 MB/s eta 0:00:00


In [3]:
import os
import subprocess

In [4]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:False"

In [5]:
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"

In [6]:
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install -q unsloth vllm human-eval tqdm
else:
    # Version matching for Colab GPUs
    try:
        import numpy, PIL
        get_numpy = f"numpy=={numpy.__version__}"
        get_pil   = f"pillow=={PIL.__version__}"
    except Exception:
        get_numpy, get_pil = "numpy", "pillow"

    try:
        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except Exception:
        is_t4 = False

    # A100/H100: vllm 0.10.2, T4: vllm 0.9.2 + pinned triton
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")

    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers tqdm human-eval
    !uv pip install -qqq {get_triton}

In [7]:
import json
import re
import ast
import torch
import random
import numpy as np
from tqdm import tqdm

from unsloth import FastLanguageModel
from vllm import SamplingParams

from human_eval.data import read_problems, write_jsonl
from human_eval.evaluation import evaluate_functional_correctness

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/torchao/quantization/quant_api.py:2525: SyntaxWarning: invalid escape sequence '\.'
  * regex for parameter names, must start with `re:`, e.g. `re:language\.layers\..+\.q_proj.weight`.


INFO 12-30 02:29:47 [__init__.py:244] Automatically detected platform cuda.
ERROR 12-30 02:29:50 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:
SEED = 3407
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8c9690ae-c44c-5dcf-cf3b-904693311f78)


# Step 2: Verifying GPU and Environment

In [9]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Torch version: 2.7.0+cu126
CUDA available: True
GPU: Tesla T4


# Step 3: Setting Up the Main Variables

In [10]:
# HumanEval evaluation settings
N_SAMPLES_PER_PROBLEM = 1          # pass@1 by default; raise to 5 or 10 if you later want pass@k
MAX_NEW_TOKENS_NON_COT = 256       # Non-CoT completions are usually short (function body)
MAX_NEW_TOKENS_COT = 768           # CoT can be longer due to tags + full function

TEMP_NON_COT = 0.0                # low sampling noise; stable for benchmarking
TEMP_COT = 0.0                     # encourages exploration under schema (optional)

TOP_P = 0.95
MIN_P = 0.10

# Stop guards (prevent rambling without clipping typical solutions)
STOP_STRINGS = ["\nclass ", "\ndef ", "\nif __name__"]

In [11]:
# Output paths (keep all artifacts under one folder)
EVAL_DIR = "data/evaluation"
os.makedirs(EVAL_DIR, exist_ok=True)

In [12]:
# Model paths / identifiers
BASE_MODEL_PATH = "unsloth/Qwen3-4B-Base"
SFT_MODEL_PATH  = "models/qwen3-4b-sft"
GRPO_MODEL_PATH = "models/qwen3-4b-grpo-final"

In [13]:
# CoT system prompt (must match training schema)
COT_SYSTEM_PROMPT_HUMANEVAL = """You are a code-generation engine.

You must output your response in the following exact format:

<START_WORKING_OUT>
Concise reasoning steps required to solve the problem.
</END_WORKING_OUT>
<SOLUTION>
Write ONLY the function body that continues from the given prompt.
Do NOT repeat the function signature (the 'def ...:' line).
Do NOT include tests, examples, explanations, or any text outside these tags.
</SOLUTION>

Do not output anything outside these tags.
"""

# Step 4: Loading HumanEval Problems

In [14]:
problems = read_problems()  # dict: {task_id: {"prompt":..., "test":..., "entry_point":...}}
task_ids = list(problems.keys())

print(f"Loaded HumanEval problems: {len(task_ids)}")
print("Example task_id:", task_ids[0])
print("\n--- Prompt Preview ---")
print(problems[task_ids[0]]["prompt"][:500])

Loaded HumanEval problems: 164
Example task_id: HumanEval/0

--- Prompt Preview ---
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



# Step 5: Prompt Builders

In [15]:
def build_non_cot_prompt(problem: dict) -> str:
    """
    Non-CoT: HumanEval-style continuation.
    We provide ONLY the HumanEval prompt.
    The harness will prepend this prompt again during execution.
    """
    return problem["prompt"]

In [16]:
def build_cot_prompt(problem: dict, tokenizer) -> str:
    """
    CoT: Uses the same chat template distribution as SFT/GRPO training.
    Returns a fully formatted ChatML prompt string.
    """
    messages = [
        {"role": "system", "content": COT_SYSTEM_PROMPT_HUMANEVAL},
        {"role": "user", "content": problem["prompt"]},
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

# Step 6: Output Post-processing Logic

In [17]:
# Stop strings (for safety against rambling)
# These are conservative: they stop the model from starting a NEW definition / class / main block.
STOP_STRINGS = ["\nclass ", "\ndef ", "\nif __name__"]

SOLUTION_RE = re.compile(r"<SOLUTION>(.*?)</SOLUTION>", re.DOTALL | re.IGNORECASE)

In [18]:
def extract_cot_solution(completion_only_text: str) -> str:
    """
    Extract code from <SOLUTION>...</SOLUTION>.
    Returns "" on failure (schema violation → harness will fail, which is correct behavior).
    """
    m = SOLUTION_RE.search(completion_only_text)
    if not m:
        return ""
    return m.group(1).strip()

In [19]:
def truncate_on_stop_strings(text: str, stop_strings: list[str]) -> str:
    """
    Stops the completion if it begins a new unrelated block (def/class/main).
    This reduces harness crashes from rambling continuations.
    """
    cut = len(text)
    for s in stop_strings:
        idx = text.find(s)
        if idx != -1:
            cut = min(cut, idx)
    return text[:cut].rstrip()

# Step 7: Defining Evaluation Loop

In [20]:
def evaluate_model(
    model_path: str,
    tokenizer,
    problems: dict,
    task_ids: list[str],
    *,
    use_cot: bool,
    output_jsonl: str,
    max_new_tokens: int,
    temperature: float,
    top_p: float = 0.95,
    min_p: float = 0.10,
    load_in_4bit: bool = False,
    gpu_memory_utilization: float = 0.9,
):
    """
    Generates 1 completion per HumanEval problem and writes JSONL for the official harness.

    JSONL schema: {"task_id": ..., "completion": ...}
    The human-eval harness will run: problem["prompt"] + completion + tests

    Therefore:
      - Non-CoT completion MUST be just the continuation (no prompt duplication).
      - CoT completion MUST be extracted code from <SOLUTION> and MUST NOT repeat signature.
    """

    print(f"\n Loading: {model_path} | Mode: {'CoT' if use_cot else 'Non-CoT'}")

    model, _ = FastLanguageModel.from_pretrained(
        model_name=model_path,
        max_seq_length=3072,
        load_in_4bit=load_in_4bit,
        fast_inference=True,
        gpu_memory_utilization=gpu_memory_utilization,
    )
    FastLanguageModel.for_inference(model)

    samples = []

    for task_id in tqdm(task_ids, desc=f"Generating ({'CoT' if use_cot else 'Non-CoT'})"):
        problem = problems[task_id]

        # 1) Build prompt
        if use_cot:
            prompt_text = build_cot_prompt(problem, tokenizer)
        else:
            prompt_text = build_non_cot_prompt(problem)

        # 2) Tokenize (IMPORTANT: always produce input_ids we can slice against)
        inputs = tokenizer(
            prompt_text,
            return_tensors="pt",
        ).to("cuda")

        input_len = inputs["input_ids"].shape[1]

        # 3) Generate
        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=False,          # deterministic benchmarking
                top_p=top_p,
                min_p=min_p,
                eos_token_id=tokenizer.eos_token_id,
                use_cache=True,
            )

        # 4) Decode ONLY newly generated tokens (completion-only)
        completion_only = tokenizer.decode(
            out[0][input_len:],
            skip_special_tokens=False,   # keep tags for CoT extraction
        )

        # 5) Post-process into harness-ready completion
        if use_cot:
            completion = extract_cot_solution(completion_only)
            # Optional stop-guard on extracted code (prevents second defs)
            completion = truncate_on_stop_strings(completion, STOP_STRINGS)
        else:
            completion = completion_only
            completion = truncate_on_stop_strings(completion, STOP_STRINGS)

        samples.append({"task_id": task_id, "completion": completion})

    write_jsonl(output_jsonl, samples)
    print(f" Saved {len(samples)} samples to: {output_jsonl}")

    # Cleanup to avoid VRAM fragmentation when you load the next model
    del model
    torch.cuda.empty_cache()